In [118]:
import numpy as np

In [119]:
class Neuron:
    def __init__(self, number_of_weights, bias, lr):
        self.weights = np.random.normal(0, 1, size=number_of_weights)
        self.bias = bias
        self.error = 0
        self.delta_weights = []
        self.delta_bias = 0
        self.lr = lr
    
    def calculate_neuron(self, inputs):
        """
        Calculate the output of this neuron
    
        """
        self.inputs = inputs
        list_outputs = []
        if len(inputs) == len(self.weights):
            for x, weight in zip(inputs, self.weights):
                list_outputs.append(x*weight)
        else:
            raise ValueError("The list of inputs and weights must have the same length, check if the neurons are well initialised")
        
        self.sum_output = sum(list_outputs)
        self.outcome = self.sum_output + self.bias
        output_sigmoid = self.sigmoid_function(self.outcome)
        return output_sigmoid
    
    def sigmoid_function(self, x):
        return 1/(1+np.exp(-x))
        
    def get_weights_and_error(self):
        return [self.weights, self.error]
    
    def update_neuron(self):
        for index in range(len(self.weights)):
            self.weights[index] -= self.delta_weights[index]
        self.bias-= self.delta_bias
        
        self.delta_weights = []
       
        #print('weights = ', self.weights, ' bias = ', self.bias, ' error = ', self.error)
            
class Output_neuron(Neuron):
    def calculate_error(self, output_neuron, target):
        self.error = output_neuron * (1 - output_neuron) * - (target - output_neuron)

    def gradient(self, input_x):
        return input_x * self.error
        
    def calculate_delta(self, output_neural_network):
        for weight, input_x in zip(self.weights, self.inputs):
            self.delta_weights.append(self.lr * self.gradient(input_x))
        self.delta_bias = self.lr * self.error
        
class Hidden_neuron(Neuron):
    def calculate_error(self, deltas_weights_right_layer):
        self.error += sum(deltas_weights_right_layer) 
        
    def calculate_delta(self): # let op dat je de error van de rechter layer pakt klopt dit?
        for weight, input_x in zip(self.weights, self.inputs):
            self.delta_weights.append(self.error * weight * input_x)
        self.delta_bias = self.bias - self.error 

In [120]:
class Neuron_layer:
    def __init__(self, neurons):
        self.neurons = neurons
        self.hidden_errors = []
    
    def calculate_layer(self, inputs):
        output = []
        for neuron in self.neurons:
            output.append(neuron.calculate_neuron(inputs))
        return output
    
    def set_output_layer(self, outputNN, target):
        for neuron in self.neurons:
            neuron.calculate_error(outputNN, target)
            neuron.calculate_delta(outputNN)
    
    def set_hidden_layer(self, weights_and_error_right_layer): 
        for index in range(len(self.neurons)):
            self.neurons[index].calculate_error([info[0][index]*info[1] for info in weights_and_error_right_layer])
            self.neurons[index].calculate_delta()
    
    def get_weights_and_error(self):
        weights_and_error_layer = []
        for neuron in self.neurons:
            weights_and_error_layer.append(neuron.get_weights_and_error())
        return weights_and_error_layer
        
    def update_layer(self):
        for neuron in self.neurons:
            neuron.update_neuron()
    
    def __str__(self):
        return f'The layer consist out of these neurons: {self.neurons}'

In [ ]:
# test Neuron layer

def test_calculate_layer():
    test_neuron1 = Hidden_neuron(2, 0, 0.1)
    test_neuron2 = Hidden_neuron(2, 0, 0.1)
    test_neuron3 = Output_neuron(2, 0, 0.1)
    
    test_layer1 = Neuron_layer([test_neuron1, test_neuron2])
    test_layer2 = Neuron_layer([test_neuron3])
    
    test_layer1.calculate_layer()
    
    assert .404  == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')


In [121]:
class Neuron_network:
    def __init__(self, neuron_layers):
        self.neuron_layers = neuron_layers
    
    def feed_forward(self, inputs):
        #calculated the network by passing in inputs and reassign the output of the first layer to inputs it passes the output of layer A to layer B
        for neuron_layer in self.neuron_layers:
            inputs = neuron_layer.calculate_layer(inputs)
        return inputs
    
    def __str__(self):
        return f'The network consist out of these perceptron_layers: {self.neuron_layers}'
    
    def total_loss(self, target_list, output_list): # loss function 
        return np.sum([(a-b)**2 for a,b in zip(target_list, output_list)]) / 2 * len(target_list)
    
    def train_nn(self, posible_inputs):
        """
        Put in like this [[[0, 0], 0],
                          [[0, 1], 0],
                          [[1, 0], 0],
                          [[1, 1], 1]]
        """
        error = 1 # The error is unknown but will be assigned after the first check
        epoch = 0
        
        error_limit_value = 0.05
        while error > error_limit_value: #epoch < 10: #
            li = []
            for posible_input in posible_inputs:
                outputNN = self.feed_forward(posible_input[0])[0]
                li.append(outputNN)
            
                # backpropagation
                self.neuron_layers[-1].set_output_layer(outputNN, posible_input[1])
                
                for left_layer, right_layer in zip(self.neuron_layers[-2::-1], self.neuron_layers[-1::-1]): # Starting at -2 because last layer is output layer
                    weights_and_error_next_layer = right_layer.get_weights_and_error()
                    left_layer.set_hidden_layer(weights_and_error_next_layer)
                
                # update
                for layer in self.neuron_layers:
                    layer.update_layer()
            
            # check the difference between outcome of the network and the targets
            error = self.total_loss([x[1] for x in posible_inputs], li)
            print('error = ', error)
            if error < error_limit_value:
                print('network trained!')
                print('epochs = ', epoch)
            
            epoch+=1
            
            # shuffle the possible inputs in order to give better learn results
            #np.random.shuffle(posible_inputs)

In [122]:
and_neuron = Output_neuron(2, 0, 0.2)

output_layer = Neuron_layer([and_neuron])

neural_network_AND = Neuron_network([output_layer])

In [123]:
neural_network_AND.train_nn([[[0, 0], 0],
                             [[0, 1], 0],
                             [[1, 0], 0],
                             [[1, 1], 1]])

error =  2.4625148836340243
error =  2.4384012241890503
error =  2.4153619726787303
error =  2.393357275019481
error =  2.3723450429389095
error =  2.352281743711919
error =  2.3331230777414405
error =  2.314824549073397
error =  2.297341936524945
error =  2.280631674660067
error =  2.264651154583915
error =  2.2493589546489403
error =  2.234715010848116
error =  2.220680736060209
error =  2.2072190965253164
error =  2.194294653054738
error =  2.181873573582203
error =  2.169923622788744
error =  2.158414133710234
error =  2.1473159654821967
error =  2.136601450699448
error =  2.126244335270739
error =  2.1162197131287535
error =  2.10650395770917
error =  2.097074651732609
error =  2.087910516503482
error =  2.0789913416729138
error =  2.070297916192203
error =  2.0618119610024555
error =  2.0535160638591083
error =  2.0453936165721105
error =  2.0374287548487557
error =  2.0296063008528775
error =  2.02191170853774
error =  2.0143310117678173
error =  2.006850775214167
error =  1.999

error =  0.30600782529504617
error =  0.3053325697033027
error =  0.30465993808456315
error =  0.30398991597220804
error =  0.303322489004652
error =  0.3026576429243715
error =  0.3019953635769407
error =  0.3013356369100799
error =  0.30067844897271256
error =  0.30002378591403756
error =  0.29937163398260813
error =  0.2987219795254232
error =  0.2980748089870292
error =  0.297430108908631
error =  0.29678786592721473
error =  0.29614806677467953
error =  0.2955106982769784
error =  0.2948757473532707
error =  0.29424320101508245
error =  0.29361304636547747
error =  0.29298527059823565
error =  0.2923598609970446
error =  0.29173680493469456
error =  0.29111608987228726
error =  0.2904977033584516
error =  0.28988163302856806
error =  0.28926786660400033
error =  0.2886563918913386
error =  0.28804719678164936
error =  0.2874402692497319
error =  0.2868355973533864
error =  0.2862331692326857
error =  0.2856329731092613
error =  0.2850349972855888
error =  0.284439230144289
error =

error =  0.15597054633639598
error =  0.15576809207264197
error =  0.1555661152416538
error =  0.1553646142437271
error =  0.1551635874860487
error =  0.15496303338266013
error =  0.15476295035442184
error =  0.15456333682897633
error =  0.15436419124071438
error =  0.15416551203073883
error =  0.15396729764682932
error =  0.15376954654340766
error =  0.15357225718150386
error =  0.15337542802872067
error =  0.1531790575591998
error =  0.15298314425358805
error =  0.15278768659900327
error =  0.15259268308900129
error =  0.15239813222354184
error =  0.15220403250895603
error =  0.15201038245791226
error =  0.15181718058938526
error =  0.15162442542862195
error =  0.15143211550710978
error =  0.15124024936254424
error =  0.15104882553879725
error =  0.15085784258588494
error =  0.15066729905993664
error =  0.15047719352316286
error =  0.15028752454382427
error =  0.1500982906962009
error =  0.14990949056056108
error =  0.14972112272313093
error =  0.14953318577606403
error =  0.14934567

error =  0.1066412413655553
error =  0.10654045775147089
error =  0.10643985036100079
error =  0.10633941875180884
error =  0.10623916248299484
error =  0.10613908111508957
error =  0.10603917421004735
error =  0.10593944133124258
error =  0.10583988204346279
error =  0.1057404959129033
error =  0.10564128250716154
error =  0.10554224139523229
error =  0.10544337214750103
error =  0.10534467433573913
error =  0.1052461475330984
error =  0.10514779131410534
error =  0.10504960525465604
error =  0.10495158893201065
error =  0.10485374192478783
error =  0.10475606381295995
error =  0.10465855417784733
error =  0.10456121260211254
error =  0.10446403866975668
error =  0.10436703196611216
error =  0.10427019207783919
error =  0.10417351859291926
error =  0.1040770111006509
error =  0.10398066919164431
error =  0.10388449245781567
error =  0.10378848049238334
error =  0.10369263288986086
error =  0.10359694924605424
error =  0.10350142915805444
error =  0.10340607222423465
error =  0.1033108

error =  0.07100566151341417
error =  0.07095848070191366
error =  0.07091135882445593
error =  0.07086429577434922
error =  0.07081729144515356
error =  0.07077034573067938
error =  0.07072345852498704
error =  0.07067662972238695
error =  0.07062985921743742
error =  0.0705831469049452
error =  0.07053649267996388
error =  0.07048989643779353
error =  0.0704433580739804
error =  0.07039687748431508
error =  0.07035045456483338
error =  0.07030408921181411
error =  0.0702577813217791
error =  0.0702115307914928
error =  0.07016533751796093
error =  0.07011920139842998
error =  0.07007312233038698
error =  0.0700271002115579
error =  0.06998113493990818
error =  0.06993522641364067
error =  0.06988937453119616
error =  0.06984357919125195
error =  0.06979784029272135
error =  0.06975215773475342
error =  0.06970653141673144
error =  0.06966096123827305
error =  0.06961544709922936
error =  0.06956998889968376
error =  0.06952458653995235
error =  0.06947923992058172
error =  0.06943394

error =  0.05361003693342432
error =  0.053582290262144235
error =  0.05355457083107726
error =  0.053526878601255753
error =  0.05349921353378506
error =  0.05347157558984318
error =  0.053443964730680554
error =  0.05341638091762005
error =  0.05338882411205671
error =  0.05336129427545788
error =  0.05333379136936239
error =  0.053306315355380926
error =  0.05327886619519581
error =  0.05325144385056068
error =  0.05322404828330032
error =  0.05319667945531055
error =  0.05316933732855838
error =  0.05314202186508106
error =  0.05311473302698701
error =  0.0530874707764543
error =  0.05306023507573188
error =  0.05303302588713854
error =  0.05300584317306304
error =  0.05297868689596372
error =  0.05295155701836882
error =  0.05292445350287579
error =  0.052897376312151474
error =  0.05287032540893176
error =  0.05284330075602173
error =  0.05281630231629488
error =  0.052789330052693664
error =  0.05276238392822896
error =  0.05273546390598018
error =  0.052708569949094494
error = 

In [124]:
inputs = [[[0,0],0],
          [[0,1],0],
          [[1,0],0],
          [[1,1],1]]

print('x1, x2,           output              expected_output')
for input_x in inputs:
    print(input_x, '   ', neural_network_AND.feed_forward(input_x[0])[0], '       ', input_x[1])

x1, x2,           output              expected_output
[[0, 0], 0]     0.0009851065344141656         0
[[0, 1], 0]     0.08537770779992837         0
[[1, 0], 0]     0.08550753271540096         0
[[1, 1], 1]     0.898492371831739         1


In [125]:
nand_neuron_x = Hidden_neuron(2, 0, 0.2)
or_neuron_x = Hidden_neuron(2, 0, 0.2)
#extra_neuron_1 = Hidden_neuron(2, 0, 0.2, 3)
#extra_neuron_2 = Hidden_neuron(2, 0, 0.2, 4)

and_neuron_x = Output_neuron(2, 0, 0.2)


hidden_layer_x = Neuron_layer([nand_neuron_x, or_neuron_x])
output_layer_x = Neuron_layer([and_neuron_x])

neural_network_xor = Neuron_network([hidden_layer_x, output_layer_x])

In [126]:
neural_network_xor.train_nn([[[0, 0], 0],
                             [[0, 1], 1],
                             [[1, 0], 1],
                             [[1, 1], 0]])

error =  2.039514115517442
error =  2.0422468620154053
error =  2.047440920280411
error =  2.0545920880623356
error =  2.062839321380282
error =  2.071031764638811
error =  2.078684616680598
error =  2.087626598166861
error =  2.101963393817608
error =  2.1265795472852984
error =  2.1637076866532956
error =  2.1986529677810243
error =  2.2087342182570975
error =  2.2059786211694146
error =  2.2021369086561515
error =  2.1981119088235452
error =  2.1939105767592437
error =  2.1895382139422885
error =  2.1850017712592904
error =  2.1803096714834296
error =  2.1754716102441534
error =  2.1704983427725537
error =  2.165401464326814
error =  2.1601931917002406
error =  2.1548861523341034
error =  2.1494931864522755
error =  2.1440271664096695
error =  2.1385008361963624
error =  2.132926672843671
error =  2.1273167703979796
error =  2.1216827462038212
error =  2.1160356684915786
error =  2.1103860037028395
error =  2.1047435816005065
error =  2.0991175759841916
error =  2.0935164987414856
e

c:\users\freek\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: overflow encountered in exp


error =  1.4519568469737896
error =  1.45074987415124
error =  1.4495685631371882
error =  1.448412195497026
error =  1.4472800762686555
error =  1.446171533169592
error =  1.4450859158258282
error =  1.444022595022586
error =  1.4429809619769782
error =  1.4419604276325142
error =  1.4409604219753027
error =  1.4399803933717443
error =  1.4390198079274352
error =  1.4380781488669638
error =  1.4371549159342414
error =  1.4362496248129593
error =  1.4353618065667573
error =  1.4344910070986376
error =  1.4336367866291717
error =  1.432798719193
error =  1.4319763921531474
error =  1.431169405732637
error =  1.4303773725629136
error =  1.4295999172485614
error =  1.428836675947814
error =  1.4280872959683577
error =  1.4273514353779275
error =  1.4266287626292102
error =  1.425918956198569
error =  1.4252217042381174
error =  1.4245367042406727
error =  1.4238636627171433
error =  1.423202294885896
error =  1.4225523243736826
error =  1.4219134829276898
error =  1.4212855101383137
error

c:\users\freek\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: overflow encountered in double_scalars
c:\users\freek\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


In [127]:
print('done')

done


In [128]:
inputs = [[[0,0],0],
          [[0,1],1],
          [[1,0],1],
          [[1,1],0]]

print('x1, x2,           output              expected_output')
for input_x in inputs:
    print(input_x, '   ', neural_network_xor.feed_forward(input_x[0])[0], '       ', input_x[1])

x1, x2,           output              expected_output
[[0, 0], 0]     nan         0
[[0, 1], 1]     nan         1
[[1, 0], 1]     nan         1
[[1, 1], 0]     nan         0


In [129]:
# test functions for class Neuron

def test_calculate_neuron():
    test_neuron = Neuron(2, 0, 0.1)
    possible_inputs = [[[0, 0], 0],
                       [[0, 1], 0],
                       [[1, 0], 0],
                       [[1, 1], 1]]
    answer = [9.357622968839299e-14, 4.5397868702434395e-05, 4.5397868702434395e-05, 0.9999546021312976]
        
    for index, inputs in zip(range(len(possible_inputs)),possible_inputs):
        test_neuron.weights = [20, 20]
        test_neuron.bias = -30
        test_output = test_neuron.calculate_neuron(inputs[0])
        assert test_output == answer[index], f'Input {possible_inputs[index]} does not match with the calculated values.'
    print('The calculate_neuron function works properly')

def test_sigmoid_function():
    test_neuron = Neuron(2, 0, 0.1)
    test_output = test_neuron.sigmoid_function(5)
    assert test_output == 0.9933071490757153, 'There is something wrong with the formule of the sigmoidfunction.'
    print('The sigmoid_function works properly')

def test_get_weights_and_error():
    test_neuron = Neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    assert test_neuron.get_weights_and_error() == [[0.5, 0.6], 0.22], 'startingvalue of delta_weights are right'
    print('The get_weights_and_error function works properly')
    
def test_update_neuron():
    possible_inputs = [[[1, 1], 1]]
    test_neuron = Neuron(2, 0, 0.1)
    test_neuron.weights = [1,1]
    test_neuron.delta_weights = [0.1, 0.2]
    test_neuron.delta_bias = 0.3
    
    test_neuron.update_neuron()
    
    assert test_neuron.weights == [0.9, 0.8], f'After the update weights should be [0.9, 0.8] instead of {test_neuron.weights}'
    assert test_neuron.bias == -0.3, f'After the update the bias should be -0.3 instead of {test_neuron.bias}'
    print('The update_neuron function works properly')

test_sigmoid_function()    
test_calculate_neuron()
test_get_weights_and_error()
test_update_neuron()

The sigmoid_function works properly
The calculate_neuron function works properly
The get_weights_and_error function works properly
The update_neuron function works properly


In [130]:
# test functions for class Output_neuron

def test_calculate_error():
    test_neuron = Output_neuron(2, 0, 0.1)
    test_neuron.calculate_error(0.3, 1)
    
    assert -0.147 == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_gradient():
    test_neuron = Output_neuron(2, 0, 0.1)
    test_neuron.error = 0.25
    test_neuron.gradient(1)
    assert 0.25 == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_calculate_delta():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    test_neuron.inputs = [1,1]
    test_neuron.calculate_delta()

    assert [0.11, 0.132] == test_neuron.delta_weights, f'Error deltaweights {test_neuron.delta_weights} does not match with [0.11, 0.32]'
    assert -0.22 == test_neuron.delta_bias, f'Error bias = {test_neuron.delta_bias} does not match with -0.22'
    print('The calculate_delta function works properly')
    
test_calculate_error()
test_gradient()
test_calculate_delta()

The calculate_error function works properly
The calculate_error function works properly
The calculate_delta function works properly


In [131]:
# test functions for class Hidden_neuron

def test_calculate_error():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.calculate_error([0.28, 0.124])
    
    assert .404  == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_calculate_delta():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    test_neuron.inputs = [1,1]
    test_neuron.calculate_delta()

    assert [0.11, 0.132] == test_neuron.delta_weights, f'Error deltaweights {test_neuron.delta_weights} does not match with [0.11, 0.32]'
    assert -0.22 == test_neuron.delta_bias, f'Error bias = {test_neuron.delta_bias} does not match with -0.22'
    print('The calculate_delta function works properly')
    
test_calculate_error()    
test_calculate_delta()

The calculate_error function works properly
The calculate_delta function works properly
